In [ ]:
# %matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import json
from mpl_toolkits.mplot3d import Axes3D
import math
# Combinatorics
from itertools import chain, combinations
import pandas as pd

# Some constants
mu_0 = 4 * np.pi * 10**(-7)     # vacuum permeability
epsilon_0 = 1 / mu_0 / 3e8**2

## Calculate Magnetic Field in a Coil

Magnetic field generated by a current flowing through a coil of given inductance

In [ ]:
def calculate_magnetic_field(inductance, current, windings, radius):
    """
    For a given inductor (inductance) and our current calculate the magnetic field
    """
    return inductance * current / (windings * np.pi * radius**2)

## Coil Inductance Estimation

There are only approximate formulas for coil inductance calculations beyond the simple solenoid configuration. I found that the one from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1456772 agrees quite well with my experimental parameters. The error is near zero for r/l = $\infty$, 0.9, 0.08 or r/l = 2.

In [ ]:
# path = r"C:\Users\GatherLab-Julian\Documents\Nextcloud\01-Studium\03-Promotion\02-Data\me-devices\rlc-settings\2021-08-20_coil-1-200uH.json"
# with open(path) as json_file:
#     rlc_settings = json.load(json_file)

# Now set the right fields
# coil_windings = float(rlc_settings["coil_windings"])
# coil_radius = float(rlc_settings["coil_radius"]) * 1e-3
# coil_length = float(rlc_settings["coil_length"]) * 1e-3

coil_windings = 45
coil_radius = 26e-3
coil_length = 16e-3

def calculate_inductance(coil_windings, coil_radius, coil_length):
    """
    Function to estimate inductance of a coil with given radius, length and number of windings
    """
    ratio = coil_radius / coil_length
    # Best for a coil with 0.9 radius to length ratio
    if math.isclose(ratio, 0.9, abs_tol = 0.3):
        return mu_0 * coil_windings**2 * coil_radius * (2.78 / (coil_length/coil_radius + 1.1) + np.log(1 + 0.39 * coil_radius/coil_length))
    # Best for a coil with 2.0 radius to length ratio
    elif math.isclose(ratio, 2, abs_tol = 0.3):
        return mu_0 * coil_windings**2 * coil_radius * (0.48 * np.log(1 + np.pi * coil_radius / coil_length) + 0.52 * np.arcsinh(np.pi * coil_radius/coil_length))
    # Otherwise take the closest and let the user know
    else:
        print("Accuracy of inductance calculation is unknown, however, closest available formula for radius/length ratio of " + str(round(ratio, 2)) + " was taken.")
        if abs(ratio - 0.9) <= abs(ratio - 2):
            return mu_0 * coil_windings**2 * coil_radius * (2.78 / (coil_length/coil_radius + 1.1) + np.log(1 + 0.39 * coil_radius/coil_length))
        elif abs(ratio - 0.9) > abs(ratio - 2):
            return mu_0 * coil_windings**2 * coil_radius * (0.48 * np.log(1 + np.pi * coil_radius / coil_length) + 0.52 * np.arcsinh(np.pi * coil_radius/coil_length))
    

inductance = calculate_inductance(coil_windings, coil_radius, coil_length)
magnetic_field_1A = calculate_magnetic_field(inductance, 1, coil_windings, coil_radius)

print("The coil inductance is estimated to be about " + str(round(inductance * 1e6, 1)) + " uH")
print("At 1 A it would provide a magnetic field of " + str(round(magnetic_field_1A * 1e3, 1)) + " mT")

# In case you want to overwrite the inductance for further calculations, uncomment
# inductance = 40e-6
# print("Overwritten with " + str(round(inductance * 1e6, 1)) + " uH")

## (Discrete) Variable Capacitor

Enter a base capacitor and a number of capacitances available to obtain the powerset of all available capacitances in your discrete capacitor matched with the resonance frequency obtained for an RLC circuit with the above inductance.

An overview of, in principle, available capacitors can be found on: https://www.rfcafe.com/references/electrical/capacitor-values.htm

In [ ]:

# In pF
# The following capacitances work perfect with an inductance of 200 uH (80 - 180 kHz) [74 kHz - 228 kHz] but the maximum frequency jump should not be higher than 1 kHz!
# For 100 kHz
# inductance = 25e-6
# base_capacitance = 18000 
# capacitances = [81, 160, 330, 680, 1200, 2200, 4700, 9100, 18000, 36000]
# For 5-40 kHz 
inductance = 700e-6
base_capacitance = 18000
capacitances = [1200, 2200, 4700, 9100, 18000, 36000, 75000, 150000, 330000, 680000]
# For 20-100 kHz 
inductance = 200e-6
# base_capacitance = 9200
# capacitances = [330, 680, 1200, 2200, 4700, 9100, 18000, 36000, 75000, 150000]
capacitances = [1200, 2200, 4700, 9100, 18000, 36000, 75000, 150000, 330000, 680000]
# For 430 kHz (3.48 mm films)
# inductance = 15e-6
# base_capacitance =2200 
# capacitances = [10, 20, 39, 81, 160, 330, 680, 1200, 2200, 4700]
# For 980 kHz (1.5 mm long films)
# inductance = 5e-6
# base_capacitance =680 
# capacitances = [5, 10, 20, 39, 81, 160, 330, 680, 1200, 2200]

#capacitances = [20, 39, 81, 160, 330, 680, 1200, 2200, 4700, 9100]  
# (Basically, by changing the base capacitance very high values can be reached)
# inductance = 51.1e-6
# base_capacitance = 1000
# capacitances = [5.1, 10, 20, 39, 81, 160, 330, 680, 1200, 2200]

# inductance = 72e-6
# base_capacitance = 2880 
# capacitances = [33, 33, 100, 220, 470, 1000]
# arduino_pin = [2, 3, 4, 5, 6, 7]

arduino_pin = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

combinations_list = list(powerset(capacitances))
combinations_pins = list(powerset(arduino_pin))

# Define pandas dataframe that contains the capacitance constituents, the corresponding arduino pins and the sum of the capacitances
# Drop if there exists more than one
combinations_df = pd.DataFrame(np.array([[list(elem) for elem in combinations_list], [list(elem) for elem in combinations_pins], np.array([np.sum(list(elem)) for elem in combinations_list]) + base_capacitance], dtype=object).T, columns=["constituents", "arduino_pins", "sum"]).sort_values("sum", ignore_index = True).drop_duplicates(subset=['sum'], keep='first')

# The capacitances can now be matched to resonance frequencies using a fit of capacitance over resonance frequency
def capacitance_to_resonance_frequency(capacitance):
    """
    Function that calculates for a given capacitance the resonance frequency (current coil with 41 windings etc.)
    Input value in pF, output value in kHz
    """
    A = 7.50279e-9
    return 1/np.sqrt(capacitance * A)

def calculate_resonance_frequency(capacitance, inductance):
    """
    For a given capacitance and inductance, calculate the resonance frequency
    """
    return np.sqrt(1 / (capacitance * 1e-12 * inductance)) / 2 / np.pi / 1e3

combinations_df["resonance_frequency"] = calculate_resonance_frequency(combinations_df["sum"].astype(np.float64).to_numpy(), inductance)
combinations_df["frequency_jump"] = abs(combinations_df["resonance_frequency"].diff())

print("Available frequency range is " + str(round(combinations_df.resonance_frequency.min(), 1)) + " - " + str(round(combinations_df.resonance_frequency.max(), 2)) + "kHz\n")
print("With a maximum frequency jump of " + str(round(combinations_df[combinations_df.resonance_frequency < 600].frequency_jump.max(), 1)))
print(combinations_df[combinations_df.frequency_jump > 1.5].to_string())
print(combinations_df.to_string())


## Coil Design Calculator (for given Inductance)

The following shall give an overview of how a coil could be reasonably designed for a provided inductance

In [ ]:
# Desired inductance
goal_inductance = 1000e-6
goal_magnetic_field = 1   # in mT

# Wire diameter
wire_diameter = 0.62e-3      # in m

# Maximum and minimum parameters
coil_radius_boundary = np.linspace(15e-3, 800e-3, 26)
# coil_length_boundary = np.linspace(10e-3, 40e-3, 31)
coil_windings = np.linspace(1, 1000, 91)

coil_configurations = pd.DataFrame(columns = ["windings", "radius", "length", "r/l", "inductance", "field_1A", "wire_length"])
i = 0

for coil_radius in coil_radius_boundary:
    # for coil_length in coil_length_boundary:
    # for coil_winding in coil_windings[coil_windings <= coil_length/wire_diameter]:
    for coil_winding in coil_windings:
        coil_length = coil_winding * wire_diameter
        # A radius over length ratio of about 0.9 should be maintained to ensure the inductance calculation formula works well.
        r_over_l = coil_radius / coil_length
        if (math.isclose(r_over_l, 0.9, abs_tol = 0.1) or math.isclose(r_over_l, 2, abs_tol = 0.3)):
            # Do the actual calculations
            calculated_inductance = calculate_inductance(coil_winding, coil_radius, coil_length)
            calculated_field_1A = calculate_magnetic_field(calculated_inductance, 1, coil_winding, coil_radius) * 1e3

            # Only save the value if the goal inductance and the goal magnetic field is matched well enough
            if (
                    #math.isclose(calculated_inductance, goal_inductance, abs_tol = 5e-6) 
                    calculated_field_1A > goal_magnetic_field * 0.5 
                ):
                coil_configurations.loc[i, "windings"] = coil_winding
                coil_configurations.loc[i, "radius"] = coil_radius * 1e3
                coil_configurations.loc[i, "length"] = coil_length * 1e3
                coil_configurations.loc[i, "r/l"] = coil_radius / coil_length
                coil_configurations.loc[i, "inductance"] = calculated_inductance * 1e6
                coil_configurations.loc[i, "field_1A"] = calculated_field_1A
                coil_configurations.loc[i, "wire_length"] = 2 * np.pi * coil_radius * coil_winding 
                # print("windings: " + str(coil_winding) + ", radius: " + str(coil_radius*1e3) + " mm, length: " + str(coil_length*1e3) + " mm")
                # print(calculated_inductance * 1e6)
                # print(calculated_field_1A)
                i += 1


print(coil_configurations.sort_values("inductance", ignore_index = True).to_string())

## Measuring magnetic fields and their frequency

Now that we can generate a magnetic field, the question is, is it right? It is very easy to measure magnetic fields using e.g. a Hall sensor. However, Hall sensors are normally used for static magnetic fields any may only be used for low frequency applications (try it out!). One easy and cost effective alternative may be a pickup coil that can be either bought commercially having military reads precision up to some MHz even or I first want to try to wind a simple one on my own and use it to measure the frequency and relative strength of my magnetic field.

### Faraday's law
```
Any change of the magnetic enviornment of a coil of wire will cause a voltage to be induced into the coil
```

The generated voltage is given by

\begin{align}
V_{gen} = - N * \frac{\Delta B \Delta A}{\Delta t}
\end{align}

which means that the higher the change in B-field or area of the coil that is penetrated by the magnetic field, and the lower the time that this change takes, the higher the generated voltage.

Example:
Coil with a diameter of 50 mm, one winding is helt in an alternating magnetic field of 2 mT at 200 kHz frequency:

\begin{align}
V_{gen} = - 1 \cdot \frac{2\cdot 10^{-3} \cdot 0.025^2\cdot\pi}{2\cdot 10^{-5}} \approx 0.785 V
\end{align}

In the case of 1 Hz frequency (turning the coil with the hand in a static field) the result is $\approx 0.24 mV$

There is definitely already an response of a coil (just a wire with one winding) visible on the oscilloscope on the smallest sensitivity level (1 mV).

In [ ]:
def calculate_magnetic_field_from_Vind(windings, radius, maximum_induced_voltage, frequency):
    """
    For a measured induced voltage calculate the magnetic field (faradays law)
    """
    return maximum_induced_voltage / (windings * (radius**2 * np.pi) * 2 * np.pi * frequency)

# Define the pickup coil
radius = 11.3e-3
windings = 1
maximum_induced_voltage = 0.26
frequency = 137e3

print(calculate_magnetic_field_from_Vind(windings, radius, maximum_induced_voltage, frequency) * 1e3)

### Helmholtz Coil Calculator

In [ ]:
def calculate_magnetic_field_helmholtz(windings, current, radius):
    """
    Magnetic field between two Helmholtz coils
    """

    return (4/5)**(3/2)*windings * current / radius * mu_0

current = 1.2
windings = 50
radius = 0.040

print(str(round(calculate_magnetic_field_helmholtz(windings, current, radius) * 1e3, 2)) + " mT magnetic field for " + str(windings) + " windings, " + str(current) + " A and " + str(radius) + " m radius")

## Calculate Capacitance of ME Device
This section is to calculate the effects of a parasitic capacitance from the ME device's third electrode due to poor isolation.

In [ ]:
# Relative permabilities
epsilon_parylene = 3
epsilon_AlOx = 10

def calculate_capacitance_plate_condensator(epsilon, area, distance):
    """
    Epsilon is relative permeability of the dielectricum
    Area is the area of the plates
    Distance is the distance between the plates (thickness of dielectricum)
    """
    return area * epsilon * epsilon_0 / distance

pad_size = 2e-3 * 5e-3
nm50_alox = calculate_capacitance_plate_condensator(epsilon_AlOx, pad_size, 50e-9)
nm200_alox = calculate_capacitance_plate_condensator(epsilon_AlOx, pad_size, 200e-9)
nm200_parylene = calculate_capacitance_plate_condensator(epsilon_parylene, pad_size, 200e-9)
nm500_parylene = calculate_capacitance_plate_condensator(epsilon_parylene, pad_size, 500e-9)
nm1000_parylene = calculate_capacitance_plate_condensator(epsilon_parylene, pad_size, 1000e-9)
nm3000_parylene = calculate_capacitance_plate_condensator(epsilon_parylene, pad_size, 3000e-9)


print("Capacitance of 50 nm AlOx between 2 x 5 mm electrodes is: " + str(round(nm50_alox * 1e12, 2)) + " pF")
print("Capacitance of 200 nm AlOx between 2 x 5 mm electrodes is: " + str(round(nm200_alox * 1e12, 2)) + " pF")
print("Capacitance of 200 nm Parylene-C between 2 x 5 mm electrodes is: " + str(round(nm200_parylene * 1e12, 2)) + " pF")
print("Capacitance of 500 nm Parylene-C between 2 x 5 mm electrodes is: " + str(round(nm500_parylene * 1e12, 2)) + " pF")
print("Capacitance of 1000 nm Parylene-C between 2 x 5 mm electrodes is: " + str(round(nm1000_parylene * 1e12, 2)) + " pF")
print("Capacitance of 3000 nm Parylene-C between 2 x 5 mm electrodes is: " + str(round(nm3000_parylene * 1e12, 2)) + " pF")

def calculate_equivalent_resistance(frequency, capacitance):
    """
    Calculate complex reactance of capacitor
    """
    return 1 / (2 * np.pi * frequency * capacitance)

print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm50_alox)) + " Ohm")
print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm200_alox)) + " Ohm")
print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm200_parylene)) + " Ohm")
print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm500_parylene)) + " Ohm")
print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm1000_parylene)) + " Ohm")
print("Equivalent ohmic resistance of capacitor is: " + str(calculate_equivalent_resistance(200e3, nm3000_parylene)) + " Ohm")